In [1]:
import pandas as pd
import requests
import os

In [2]:
# you can get the chain list straihgt from the API
url = 'https://api.llama.fi/v2/chains'
response = requests.get(url)
if response.status_code == 200: # data was fetched successfully
    data = response.json()
    df = pd.DataFrame(data)
else:
    raise Exception('Error fetching data from DeFiLlama API')

dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
dex_all_chains_response = requests.get(dex_all_chains_url)

dex_all_chains_data = dex_all_chains_response.json()
dex_all_chains = set(dex_all_chains_data.get('allChains', []))


fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
fees_all_chains_response = requests.get(fees_all_chains_url)

fees_all_chains_data = fees_all_chains_response.json()
fees_all_chains = set(fees_all_chains_data.get('allChains', []))

common_chains = dex_all_chains.intersection(fees_all_chains)

df = df[['name', 'tvl']]
df = df[df['name'].isin(common_chains)]

# Sort the data to get the top 10 chains by TVL
df = df.sort_values(by='tvl', ascending=False).head(10)
print(df)

base_folder = './data'
fee_folder = os.path.join(base_folder, 'fee')
tvl_folder = os.path.join(base_folder, 'tvl')
volume_folder = os.path.join(base_folder, 'volume')

for folder in [fee_folder, tvl_folder, volume_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

         name           tvl
15   Ethereum  6.562908e+10
46     Solana  8.739363e+09
90       Tron  7.555194e+09
6         BSC  5.188040e+09
52    Bitcoin  4.136139e+09
10       Base  3.380602e+09
8    Arbitrum  3.011868e+09
12  Avalanche  1.397563e+09
93        Sui  1.372728e+09
57      Aptos  1.107875e+09


In [3]:
def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            csv_filename = os.path.join(tvl_folder, f"{chain_name}_{metric}.csv")
            df = df[['date', metric]].tail(365)
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(365)
            csv_filename = os.path.join(volume_folder, f"{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            df = df[['date', metric]].tail(365)
            csv_filename = os.path.join(fee_folder, f"{chain_name}_{metric}.csv")
            df.to_csv(csv_filename, index=False)
            print(f"Saved {metric} data for {chain_name} to {csv_filename}")
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')

In [4]:
for chain_name in df['name']:
    for metric in ['tvl', 'volume', 'fee']:
        fetch_historical_data(chain_name, metric)

Saved tvl data for Ethereum to ./data\tvl\Ethereum_tvl.csv
Saved volume data for Ethereum to ./data\volume\Ethereum_volume.csv
Saved fee data for Ethereum to ./data\fee\Ethereum_fee.csv
Saved tvl data for Solana to ./data\tvl\Solana_tvl.csv
Saved volume data for Solana to ./data\volume\Solana_volume.csv
Saved fee data for Solana to ./data\fee\Solana_fee.csv
Saved tvl data for Tron to ./data\tvl\Tron_tvl.csv
Saved volume data for Tron to ./data\volume\Tron_volume.csv
Saved fee data for Tron to ./data\fee\Tron_fee.csv
Saved tvl data for BSC to ./data\tvl\BSC_tvl.csv
Saved volume data for BSC to ./data\volume\BSC_volume.csv
Saved fee data for BSC to ./data\fee\BSC_fee.csv
Saved tvl data for Bitcoin to ./data\tvl\Bitcoin_tvl.csv
Saved volume data for Bitcoin to ./data\volume\Bitcoin_volume.csv
Saved fee data for Bitcoin to ./data\fee\Bitcoin_fee.csv
Saved tvl data for Base to ./data\tvl\Base_tvl.csv
Saved volume data for Base to ./data\volume\Base_volume.csv
Saved fee data for Base to ./da